# **Procedimiento para realizar consultas (requests) a la base de datos de EDAMAM mediante las API's :**

## **1. Recipe Search**
## **2. Nutrition Analysis**
## **3. Food Database**         

## *Por: Samuel Cortez / Maestría en Ciencia de Datos*

In [3]:
# Importar librerías
import pandas as pd
import requests
import json         
import logging

In [95]:
# Se define clase Edamam_mcd
class Edamam_mcd:
    "API que regresa datos de tipo json"
    def __init__(self, 
                id_nutrition, 
                key_nutrition, 
                id_recipes,
                key_recipes,
                id_food,
                key_food):
        
        # Asignación de claves / ID de las API's
        self.id_nut = id_nutrition
        self.key_nut = key_nutrition
        self.id_rec = id_recipes
        self.key_rec = key_recipes
        self.id_food = id_food
        self.key_food = key_food
    
    def Nut_Analysis(self, query):
        self.Nut_ingredients = query
        # Se define url (Access Point) para realizar consulta a la API Nutrition Analysis
        url_req_nutr = 'https://api.edamam.com/api/nutrition-data'
        # Se definen los parametros a pasar por la función get para realizar consulta
        params_nutr= {
            'app_id':self.id_nut,
            'app_key':self.key_nut,
            'ingr':self.Nut_ingredients
            }
        # Envío de solicitud para realizar consulta de análisis nutricional
        Response = requests.get(url_req_nutr, params = params_nutr)

        # Imprimir el código del estatus
        print("Status Code (Nutrition Analysis API):", Response.status_code)
        
        if Response.status_code == 401:
            logging.error('{key} - Clave inválida (Nutrition Analysis API)'.format(key=self.key_nut))

        # Valor de retorno de tipo diccionario, Response tiene contenido JSON serializado
        self.r_Nut = Response.json()
        return self.r_Nut

    def Search_recipe(self, query):
        self.query_recipe = query
        # Se construye url con los parámetros para realizar consulta para busqueda de recetas de un platillo
        url_recipe = 'https://api.edamam.com/search?q={query}&app_id={id}&app_key={key}'.format(id=self.id_rec, key=self.key_rec,query=self.query_recipe)    
        Response = requests.get(url_recipe)

        # Imprimir el código de estatus
        print("Status Code (Recipe Search API) :", Response.status_code)
        
        if Response.status_code == 401:
            logging.error('{key} - Clave inválida (Recipe Search API)'.format(key=self.key_rec))

        # Valor de retorno de tipo diccionario, Response tiene contenido JSON serializado 
        self.r_Recipe = Response.json()
        return self.r_Recipe

    def Search_food(self, query):
        self.query_food = query
        # Se define url (Access Point) para realizar consulta de análisis de comida
        url_food = 'https://api.edamam.com/api/food-database/parser?'
        # Se definen los parametros a pasar por la función get para realizar consulta
        params_food = {
            'app_id':self.id_food,
            'app_key':self.key_food,
            'ingr':self.query_food
            }
        # Se realiza una consulta dando la url
        Response = requests.get(url_food, params= params_food)
        
        # Imprimir el código de estatus
        print("Status Code (Food Database API):", Response.status_code)

        if Response.status_code == 401:
            logging.error('{key} - Clave inválida (Food Database API)'.format(key=self.key_rec))

        # Valor de retorno de tipo diccionario, Response tiene contenido JSON serializado
        self.r_Food = Response.json()
        return self.r_Food
    
    # Conjunto de funciones para generar dataframes con los datos principales de las consultas

    def Nutrient_Guide(self):
        # Manejo de excepciones
        try:
            # Se crean DataFrames de las variables totalNutrients, totalDaily y totalNutrientsKCal 
            self.df_Nutrition= pd.DataFrame(self.r_Nut.get('totalNutrients')).T.rename_axis(str(self.Nut_ingredients))
            self.df_totalDaily = pd.DataFrame(self.r_Nut.get('totalDaily')).T.rename_axis(str(self.Nut_ingredients))
            self.df_total_Nut =  pd.DataFrame(Response_Nut.get('totalNutrientsKCal')).T.rename_axis(str(self.Nut_ingredients))
            # Se obtiene las calorías del alimento
            self.Nutrient_Cal = self.r_Nut.get('calories')
            # Se obtiene el peso total del alimento
            self.totalWeight = self.r_Nut.get('totalWeight')
            
        except Exception as e:
            print(e)
        
    def ingredients_table(self):
        # Manejo de excepciones
        try:
            # Comprensión de listas: se crean nuevas listas con las etiquetas de la receta y con los ingredientes
            list_label = [element_hits.get('recipe').get('label') for element_hits in Response_Recipe.get('hits')]
            list_ingredientes = [element_ingre.get('recipe').get('ingredients') for element_ingre in Response_Recipe.get('hits')]
            # Se construye DataFrame 
            df_ = pd.DataFrame(list_ingredientes, index=list_label).stack().apply(pd.Series).drop(columns=['foodId'])
            self.df_Recipe = df_.rename_axis(["Types", "Items"], axis = "rows").rename(columns = {"foodCategory": "food category", "text":"ingredient name"})

        except Exception as e:
            print(e)

    def food_table(self):
        # Se generar listas vacias
        list_foods = []
        list_nutrients = []
        rename_nutrients = ['ENERGY (kcal)', 'PROTEIN (g)', 'FAT (g)', 'CARBS (g)', 'FIBER (g)']
        # Manejo de excepciones
        try:
            # Comprensión de listas: se crean nuevas listas con las etiquetas de los alimentos y con los nutrientes  
            list_foods = [element.get('food').get('label') for  element in self.r_Food['hints']]
            list_nutrients = [element.get('food').get('nutrients') for element in self.r_Food['hints']]
            # Se construye DataFrame
            self.df_food_table = pd.DataFrame(list_nutrients, index = list_foods).round(2).fillna('Unknown')
            self.df_food_table.columns = rename_nutrients
            
        except Exception as e:
            print(e) 
    
    def write_files(self):
        #  Exportar archivos .CSV/.xlsx
        path_Nutrition = '{Nut}_Nutritional_Analysis.xlsx'.format(Nut=self.Nut_ingredients)
        path_Recipe = '{Recipe}_Recipe.csv'.format(Recipe = self.query_recipe)
        path_food = '{food}_food.csv'.format(food = self.query_food)

        # Se exporta a excel el análisis nutricional 
        with pd.ExcelWriter(path_Nutrition) as writer:
            self.df_Nutrition.to_excel(writer, sheet_name= 'Nutritional_Analysis')
            self.df_totalDaily.to_excel(writer, sheet_name='Total_Daily')
            self.df_total_Nut.to_excel(writer, sheet_name='totalNutrientsKCal')
        # Se exporta en formato CSV los ingredientes de las recetas
        self.df_Recipe.to_csv(path_Recipe)
        # Se exporta en CSV el dataframe con el análisis de los alimentos
        self.df_food_table.to_csv(path_food)
        

In [96]:
# Claves / ID's de las aplicaciones asignadas
nutrition_appid= '5f1e7888'
nutrition_appkey= '5576413f5acb38f5259a58dad888d1b9'
recipes_appid = 'dcda2d5e'
recipes_appkey= '87ae2d34cdde47026fc341c8fbd7957f'
food_appid= '6b69c52c'
food_appkey= 'd5a448b534a0c358dfb47fa7a6548724'

# Se crea el objeto 
EDAMAM_consulta = Edamam_mcd(id_nutrition=nutrition_appid, 
                            key_nutrition= nutrition_appkey,
                            id_recipes=recipes_appid,
                            key_recipes=recipes_appkey,
                            id_food=food_appid,
                            key_food=food_appkey)

# Ejercución de los métodos de cada una de las API's para realizar consultas
Response_Nut = EDAMAM_consulta.Nut_Analysis(query= '1 egg')
Response_Food = EDAMAM_consulta.Search_food(query= 'pizza')
Response_Recipe = EDAMAM_consulta.Search_recipe(query= 'ribs')
# Ejecución de los métodos para obtener los dataframe con los resultados para cada una de las API's
EDAMAM_consulta.Nutrient_Guide()
EDAMAM_consulta.ingredients_table()
EDAMAM_consulta.food_table()
# Se exportan en archivos csv y xlsx los resultados obtenidos
EDAMAM_consulta.write_files()


Status Code (Nutrition Analysis API): 200
Status Code (Food Database API): 200
Status Code (Recipe Search API) : 200


In [97]:
# Consulta a la API: Nutrition Analysis obtenida en formato json
Response_Nut

{'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_d42dd2c3fa8d4f288e4a0711a861bc73',
 'calories': 61,
 'totalWeight': 43.0,
 'dietLabels': ['LOW_CARB', 'LOW_SODIUM'],
 'healthLabels': ['LOW_FAT_ABS',
  'SUGAR_CONSCIOUS',
  'LOW_POTASSIUM',
  'KIDNEY_FRIENDLY',
  'KETO_FRIENDLY',
  'VEGETARIAN',
  'PESCATARIAN',
  'PALEO',
  'SPECIFIC_CARBS',
  'MEDITERRANEAN',
  'DAIRY_FREE',
  'GLUTEN_FREE',
  'WHEAT_FREE',
  'MILK_FREE',
  'PEANUT_FREE',
  'TREE_NUT_FREE',
  'SOY_FREE',
  'FISH_FREE',
  'SHELLFISH_FREE',
  'PORK_FREE',
  'RED_MEAT_FREE',
  'CRUSTACEAN_FREE',
  'CELERY_FREE',
  'MUSTARD_FREE',
  'SESAME_FREE',
  'LUPINE_FREE',
  'MOLLUSK_FREE',
  'ALCOHOL_FREE',
  'NO_OIL_ADDED',
  'NO_SUGAR_ADDED',
  'SULPHITE_FREE',
  'FODMAP_FREE',
  'KOSHER'],
 'cautions': [],
 'totalNutrients': {'ENERC_KCAL': {'label': 'Energy',
   'quantity': 61.49,
   'unit': 'kcal'},
  'FAT': {'label': 'Fat', 'quantity': 4.0893, 'unit': 'g'},
  'FASAT': {'label': 'Saturated', 'quantity': 1.34418, 'un

In [98]:
# Consulta a la API: Food Database obtenida en formato json
Response_Food

{'text': 'pizza',
 'parsed': [{'food': {'foodId': 'food_at830s9amds32fb8w6ufmaopzk8n',
    'label': 'Pizza',
    'nutrients': {'ENERC_KCAL': 268.0,
     'PROCNT': 10.36,
     'FAT': 12.28,
     'CHOCDF': 29.02,
     'FIBTG': 2.2},
    'category': 'Generic foods',
    'categoryLabel': 'food',
    'image': 'https://www.edamam.com/food-img/c01/c0150280d71059c23c025f501f502dc0.jpg'}}],
 'hints': [{'food': {'foodId': 'food_at830s9amds32fb8w6ufmaopzk8n',
    'label': 'Pizza',
    'nutrients': {'ENERC_KCAL': 268.0,
     'PROCNT': 10.36,
     'FAT': 12.28,
     'CHOCDF': 29.02,
     'FIBTG': 2.2},
    'category': 'Generic foods',
    'categoryLabel': 'food',
    'image': 'https://www.edamam.com/food-img/c01/c0150280d71059c23c025f501f502dc0.jpg'},
   'measures': [{'uri': 'http://www.edamam.com/ontologies/edamam.owl#Measure_unit',
     'label': 'Whole'},
    {'uri': 'http://www.edamam.com/ontologies/edamam.owl#Measure_pizza',
     'label': 'Pizza'},
    {'uri': 'http://www.edamam.com/ontologies/

In [99]:
# Consulta a la API: Recipe Search obtenida en formato json
Response_Recipe

{'q': 'ribs',
 'from': 0,
 'to': 10,
 'more': True,
 'count': 7000,
 'hits': [{'recipe': {'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_78268a10331b96aadb1afeb066ae8fdb',
    'label': 'Tender Grilled Short Ribs Recipe',
    'image': 'https://www.edamam.com/web-img/4b2/4b2e7953ff6f99118bd1c418d249bc87.jpg',
    'source': 'Serious Eats',
    'url': 'http://www.seriouseats.com/recipes/2012/05/tender-grilled-short-ribs.html',
    'shareAs': 'http://www.edamam.com/recipe/tender-grilled-short-ribs-recipe-78268a10331b96aadb1afeb066ae8fdb/ribs',
    'yield': 4.0,
    'dietLabels': ['Low-Carb'],
    'healthLabels': ['Sugar-Conscious',
     'Keto-Friendly',
     'Paleo',
     'Dairy-Free',
     'Gluten-Free',
     'Wheat-Free',
     'Egg-Free',
     'Peanut-Free',
     'Tree-Nut-Free',
     'Soy-Free',
     'Fish-Free',
     'Shellfish-Free',
     'Pork-Free',
     'Crustacean-Free',
     'Celery-Free',
     'Mustard-Free',
     'Sesame-Free',
     'Lupine-Free',
     'Mollusk-Free'

## **Despliegue de los DataFrames de la API Nutrition Analysis**

In [100]:
# Desplegar DataFrame de Nutrientes totales en kCal
EDAMAM_consulta.df_total_Nut

,label,quantity,unit
1 egg,,,
ENERC_KCAL,Energy,61,kcal
PROCNT_KCAL,Calories from protein,22,kcal
FAT_KCAL,Calories from fat,38,kcal
CHOCDF_KCAL,Calories from carbohydrates,1,kcal


In [101]:
# Desplegar DataFrame de nutrientes totales 
EDAMAM_consulta.df_Nutrition 

,label,quantity,unit
1 egg,,,
ENERC_KCAL,Energy,61.49,kcal
FAT,Fat,4.0893,g
FASAT,Saturated,1.34418,g
FATRN,Trans,0.01634,g
FAMS,Monounsaturated,1.57294,g
FAPU,Polyunsaturated,0.82173,g
CHOCDF,Carbs,0.3096,g
FIBTG,Fiber,0,g
SUGAR,Sugars,0.1591,g


In [102]:
# Desplegar DataFrame de nutrientes total diario
EDAMAM_consulta.df_totalDaily 

,label,quantity,unit
1 egg,,,
ENERC_KCAL,Energy,3.0745,%
FAT,Fat,6.29123,%
FASAT,Saturated,6.7209,%
CHOCDF,Carbs,0.1032,%
FIBTG,Fiber,0,%
PROCNT,Protein,10.8016,%
CHOLE,Cholesterol,53.32,%
NA,Sodium,2.54417,%
CA,Calcium,2.408,%


In [103]:
# Registro de Calorias
print('Calorías:',EDAMAM_consulta.Nutrient_Cal)

Calorías: 61


In [104]:
# Registro del peso total en gramos/ en este peso se basa todo el análisis nutricional
print('Peso total: {wtotal} g:'.format(wtotal = EDAMAM_consulta.totalWeight))

Peso total: 43.0 g:


## **Despliegue de los DataFrames de la API Food Database**

In [105]:
# Desplegar DataFrame de la base de datos de alimentos consultados
EDAMAM_consulta.df_food_table

,ENERGY (kcal),PROTEIN (g),FAT (g),CARBS (g),FIBER (g)
Pizza,268.00,10.36,12.28,29.02,2.20
Cheese Pizza,268.00,10.36,12.28,29.02,2.20
Frozen Pizza,268.00,10.36,12.28,29.02,2.20
Sausage Pizza,280.00,11.50,12.35,30.62,2.30
Pepperoni Pizza,282.00,11.74,11.91,31.98,2.30
Stuffed Cheeze Pizza,274.00,12.23,11.63,30.00,1.70
Veggy Cheese Pizza,244.00,11.02,10.90,25.38,2.20
Thick Crust Sausage Pizza,282.00,11.06,12.94,30.36,2.30
Thin Crust Cheese Pizza,302.00,12.85,13.95,31.20,2.50
Thick Crust Cheese Pizza,271.00,10.81,10.54,33.17,2.20


## **Despliegue de los DataFrames de la API Recipe Search**

In [93]:
# Desplegar DataFrame para la búsqueda de recetas
EDAMAM_consulta.df_Recipe

ingredient name  \
Types                                 Items                                                      
Tender Grilled Short Ribs Recipe      0      2 pounds boneless short ribs or 4 pounds Engli...   
                                      1            Kosher salt and freshly ground black pepper   
                                      2            Kosher salt and freshly ground black pepper   
Sticky glazed ribs                    0               2kg spare ribs, cut into individual ribs   
                                      1                              2 tbsp barbecue spice mix   
...                                                                                        ...   
Molasses-Glazed Cocktail Ribs recipes 3                                  1/2 cup Dijon mustard   
                                      4                                        1/4 cup harissa   
                                      5      2 pounds baby back or spare ribs, cut in half ...   
                                      6                                    ground black pepper   
                                      7                                            coarse salt   

                                             quantity     measure  \
Types                                 Items                         
Tender Grilled Short Ribs Recipe      0          2.00       pound   
                                      1          0.00        None   
                                      2          0.00        None   
Sticky glazed ribs                    0          2.00    kilogram   
                                      1          2.00  tablespoon   
...                                               ...         ...   
Molasses-Glazed Cocktail Ribs recipes 3          0.50         cup   
                                      4          0.25         cup   
                                      5          2.00       pound   
                                      6          0.00        None   
                                      7          0.00        None   

                                                            food       weight  \
Types                                 Items                                     
Tender Grilled Short Ribs Recipe      0      boneless short ribs   907.184740   
                                      1              Kosher salt     5.443108   
                                      2             black pepper     2.721554   
Sticky glazed ribs                    0                     ribs  2000.000000   
                                      1                spice mix     5.400000   
...                                                          ...          ...   
Molasses-Glazed Cocktail Ribs recipes 3            Dijon mustard   124.500000   
                                      4                  harissa    61.250000   
                                      5                     ribs   907.184740   
                                      6             black pepper     4.214429   
                                      7              coarse salt     8.428858   

                                                     food category  \
Types                                 Items                          
Tender Grilled Short Ribs Recipe      0                      meats   
                                      1      Condiments and sauces   
                                      2      Condiments and sauces   
Sticky glazed ribs                    0                      meats   
                                      1      Condiments and sauces   
...                                                            ...   
Molasses-Glazed Cocktail Ribs recipes 3      Condiments and sauces   
                                      4                canned soup   
                                      5                      meats   
                                      6      Condiments and sauces   
            